In [5]:
import pandas as pd
import numpy as np
import datetime
import s3fs
import numcodecs as ncd
import xarray as xr
import cartopy.crs as ccrs

# Setup
startdate = "2023-10-01"
enddate = "2023-10-02"
point_lat = 39.58148838130895
point_lon = -105.94259648925797
level = 'surface'
fs = s3fs.S3FileSystem(anon=True)
projection = ccrs.LambertConformal(central_longitude=262.5, central_latitude=38.5, 
                                   standard_parallels=(38.5, 38.5),
                                   globe=ccrs.Globe(semimajor_axis=6371229, semiminor_axis=6371229))

x, y = projection.transform_point(point_lon, point_lat, ccrs.PlateCarree())
chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
nearest_point = chunk_index.sel(x=x, y=y, method="nearest")

# Data Retrieval Function
def retrieve_data(s3_url, var):
    with fs.open(s3_url, 'rb') as compressed_data:
        buffer = ncd.blosc.decompress(compressed_data.read())
        dtype = "<f4" if "surface/PRES" in s3_url else "<f2"
        chunk = np.frombuffer(buffer, dtype=dtype)
        data_array = np.reshape(chunk, (len(chunk)//(150*150), 150, 150))
        return data_array

# Data Collection
data_collection = []
variables = ["SNOD", "GUST", "ASNOW_acc_fcst", "PRES", "TMP"]

for day in pd.date_range(start=startdate, end=enddate):
    for hour in range(24):
        date_str = day.strftime('%Y%m%d')
        hour_str = f'{hour:02d}'
        row = {'date': date_str, 'hour': hour_str}
        for var in variables:
            data_url = f'hrrrzarr/sfc/{date_str}/{date_str}_{hour_str}z_fcst.zarr/{level}/{var}/{level}/{var}/'
            fcst_chunk_id = f"0.{nearest_point.chunk_id.values}"
            data = retrieve_data(data_url + fcst_chunk_id, var)
            gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
            row[var] = gridpoint_forecast.mean()
        data_collection.append(row)

# Convert to DataFrame and Export to CSV
df = pd.DataFrame(data_collection)
df.to_csv('historical_hrrr_fcst.csv', index=False)